# 🚀 **Arize and Feast Walkthrough**

Arize and Feast are two platforms aimed at different, but connected, parts of the ML pipeline. Arize helps you visualize your model performance, understand drift & data quality issues, and share insights as your **Evaluation Store**. Feast (i.e, **Feature Store**) is an operational data system for managing and serving machine learning features to models in production.

# ✨ Prologue: The Data Consistency Story
You are building a transaction fraud model that predicts whether a payment by a customer succeed. Two of your most important features are `current_24_hr_cancels` and `avg_cancels` referring to the most up-to-date information about cancellations occurring across transactions. 

In this example, we will show how data inconsistency can occur based on arrival of new information. The **online features** are features served in production, usually "materialized" live in production for low latency. The **offline features** are features used retraining models, often retrieved later after thos features are already served in production.

However, there is typically a delay between getting the online features and when the data sources from which those features are updated. For example, there could be a delay in parquet data source registering a cancellation for our transactions.

This delay causes discrepancies between online and offline environments for `current_24_hr_cancels` and `avg_cancels`, since these features are computed assuming the logs from our parquet file is up-to-date.


In this tutorial, we will show how to catch this using **Feast with Arize:** surfacing up and when they are inconsistent, how are they inconsistent, and what can do the troubleshoot once we understand the problem. 

## ✔️ Steps for this Walkthrough

1. Install Dependencies & Set-up Feast + Arize Repo/Credentials
2. Online: `store.get_online_features` from feast and `arize.log` Integration 
3. Offline: `store.get_historical_features` from feast and `arize.log_validation_records` Integration
4. On Arize: Measuring Data Inconsistency
5. On Arize: Troubleshooting data consistency and why this matters

# **Step 1: Install Dependencies & Set-up Feast + Arize Repo/Credentials**
In this section, we will set-up the Colab environment so that Feast works properly in under 1 minute!

## **Step 1.1: Installing Dependencies**

In [ ]:
from IPython.display import clear_output
import os

!pip install feast -q
!pip install arize -q

clear_output(wait=True)
print("✅ Packages has been installed. ⚠️ Restart run-time before running next cell!")

✅ Packages has been installed. ⚠️ Restart run-time before running next cell!


##**Step 1.2: ⚠️ Restart Run-time** ⚠️
Since Colab comes with pre-installed environments, you will need to reset some dependencies following these steps:

1. Click on `Runtime`
2. Click on `Re-start Runtime` (Don't click the other options!)

Then you can continue the tutorial from the cell below (no need to run pip install again). 

If you ever need to reset the tutorial, you can run this cell again.


In [ ]:
try:
    import feast
except:
    raise ValueError("⚠️ You need to restart run-time to reset Colab dependencies")
from IPython.display import clear_output
import os

# setting up the repo
if 'sample_data' in os.listdir():
    !git clone https://github.com/alanschen/feast_example.git
    %cd feast_example

!python reset_environment.py # this resets your feast environment
clear_output(wait=True)

if 'feature_store.yaml' in os.listdir():
    print("✅ Environment has been re-set properly")

✅ Environment has been re-set properly


## **Step 1.3: Set-up Feature Store & Materialize Feature**

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# 1 Defining a FeatureStore
from feast import FeatureStore
store = feast.FeatureStore(repo_path=".")

# 2 Applying feast definitions to the online registry : registry.db
print("✅ Applying feast definition to registry.db")
!feast apply

# 3 materializing features into our online sqlite feature store
target_time = datetime(2021, 6, 1, 22, 59, 59)
TARGET_TIME = target_time.strftime("%Y-%m-%dT%H:%M:%S")
print("✅ Materializing one feature into online_store.db")
!feast materialize '2021-06-01T22:50:00' $TARGET_TIME

# 2 Defining our feature references:
print("\n✅ Showing an example entries in our `data_source.parquet`")
cancel = pd.read_parquet('data/cancel_stats.parquet')
cancel

✅ Applying feast definition to registry.db
Registered entity user_id
Registered feature view user_cancel_stats
Deploying infrastructure for user_cancel_stats
✅ Materializing one feature into online_store.db
Materializing 1 feature views from 2021-06-01 22:50:00+00:00 to 2021-06-01 22:59:59+00:00 into the sqlite online store.

user_cancel_stats:
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 25.24it/s]

✅ Showing an example entries in our `data_source.parquet`


,datetime,user_id,current_24_hr_cancels,avg_cancels,created
0,2021-06-01 22:45:12,501,5,5.742,2021-06-01 22:45:12
1,2021-06-01 22:47:23,501,5,5.742,2021-06-01 22:47:23
2,2021-06-01 22:49:24,501,5,5.742,2021-06-01 22:49:24
3,2021-06-01 22:51:45,501,6,5.742,2021-06-01 22:51:45
4,2021-06-01 22:52:03,501,6,6.323,2021-06-01 22:52:03
5,2021-06-01 22:55:50,501,7,6.323,2021-06-01 22:55:50


## **Step 1.4: Set-up Arize Client**
First, copy the Arize `API_KEY` and `SPACE_KEY` from your admin page linked below!



In [ ]:
from arize.api import Client
from arize.types import ModelTypes
import concurrent.futures as cf


# Step 1.2: Set-up Arize Client and model meta data
SPACE_KEY = 'SPACE_KEY'
API_KEY = 'API_KEY'
arize = Client(space_key=SPACE_KEY, api_key=API_KEY)

model_id = 'arize-feast-tutorial'
model_version = '1.0'
model_type = ModelTypes.SCORE_CATEGORICAL

if SPACE_KEY == 'SPACE_KEY' or API_KEY == 'API_KEY': 
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else: 
    print("✅ Arize setup complete!")

✅ Arize setup complete!


#**Step 2: Online Environment Integration**
After running `feast materialize` in the previous step, the most recent feature for `user_id: 501` has been stored to `online_store.db`. Then, by running `store.get_online_feature`, we can fetch the feature, predict outcomes with our best model, and use `arize.log` to record evaluations to the Arize as your Evaluation Store.

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://arize.gitbook.io/arize/apis/python-sdk-1/arize.log)

In [ ]:
import uuid

# define online feature
feature_refs = ["user_cancel_stats:current_24_hr_cancels", "user_cancel_stats:avg_cancels"]
online_data = store.get_online_features(
    feature_refs=feature_refs, # as defined above
    entity_rows=[{"user_id": 501}],
)

feature_labels = ['user_cancel_stats__current_24_hr_cancels', 'user_cancel_stats__avg_cancels']

# formatting the feature into a dataframe
print("\n✅ Feast online feature materialized and retrieved")
display(online_data.to_df()[feature_labels])

# logging to arize
prediction_id = '1a43c711-f407-4a5c-aa54-c22344a9879c' # we will need this to join the records
prediction_label = 'fraud'
actual_label = 'fraud'
prediction_timestamp = int(target_time.timestamp()) # this will be used to create match environment on Arize

log_result = arize.log(model_id=model_id,
          model_version=model_version,
          prediction_id=prediction_id,
          prediction_label=prediction_label,
          prediction_timestamp=prediction_timestamp,
          features=online_data.to_dict()
          )
assert log_result.result().status_code == 200, "Logging should be successful"

print("\n✅ Arize online feature logging complete")


✅ Feast online feature materialized and retrieved


,user_cancel_stats__current_24_hr_cancels,user_cancel_stats__avg_cancels
0,7,6.323



✅ Arize online feature logging complete


# **Step 3: Offline Environment Integration**
**Important**: Notice for `event_timestamp` we used the `target_time` again, so we are using the **exact same timestamp** as when we called `get_online_feature` when calling `get_historical_features`.

Next, we can then log the offline data to Arize as validation using `arize.log_validation_records` when we are re-training our model using the historical/offline features. Click below for the documentations.

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://arize.gitbook.io/arize/apis/python-sdk-1/arize.log_validation_records)

In [ ]:
# In even the most performant real time systems, the most up to date information about the world
# takes time to fully propagate throughout the stack. In this case, we simulate a scenario where 
# the most recent information about user 501's cancel behavor rolls up to the feature store 
# AFTER the prediction has been made with the stale information.
latest_arrival_time = datetime(2021, 6, 1, 22, 56, 30)

# Add an entry to our data source to simulate new entry in data source
cancel.loc[6] = [latest_arrival_time, 501, 8, 7.127, latest_arrival_time]
cancel.to_parquet("data/cancel_stats.parquet")

print("⚠️ A new data entry logged into the data source parquet:")
display(cancel)

⚠️ A new data entry logged into the data source parquet:


,datetime,user_id,current_24_hr_cancels,avg_cancels,created
0,2021-06-01 22:45:12,501,5,5.742,2021-06-01 22:45:12
1,2021-06-01 22:47:23,501,5,5.742,2021-06-01 22:47:23
2,2021-06-01 22:49:24,501,5,5.742,2021-06-01 22:49:24
3,2021-06-01 22:51:45,501,6,5.742,2021-06-01 22:51:45
4,2021-06-01 22:52:03,501,6,6.323,2021-06-01 22:52:03
5,2021-06-01 22:55:50,501,7,6.323,2021-06-01 22:55:50
6,2021-06-01 22:56:30,501,8,7.127,2021-06-01 22:56:30


In [ ]:
# Retrieving offline data to Arize
entity_df = pd.DataFrame.from_dict({"user_id": [501], "event_timestamp": [target_time]})
offline_data = store.get_historical_features(
    entity_df=entity_df,
    feature_refs=feature_refs,
).to_df().sort_values('user_id')

print("\n✅ Feast offline (historical) feature retrieved")
# logging validation data to Arize
arize.log_validation_records(
    model_id=model_id,
    model_version=model_version,
    batch_id='offline',
    prediction_labels=pd.Series([prediction_label]), # we need to pass in the same prediction_labels
    actual_labels=pd.Series([actual_label]),
    prediction_ids=pd.Series([prediction_id]),
    prediction_timestamps=pd.Series([prediction_timestamp]) # and the same timest
)
print("\n✅ Arize offline (validation) features logging complete")

print("\n⚠️ Notice a mismatch between online environment")
display(online_data.to_df()[feature_labels])

print("\n⚠️ ... and offline environment!")
display(offline_data[feature_labels])


✅ Feast offline (historical) feature retrieved

✅ Arize offline (validation) features logging complete

⚠️ Notice a mismatch between online environment


,user_cancel_stats__current_24_hr_cancels,user_cancel_stats__avg_cancels
0,7,6.323



⚠️ ... and offline environment!


,user_cancel_stats__current_24_hr_cancels,user_cancel_stats__avg_cancels
0,8,7.127


# **Step 4: Arize Platform - Measuring Data Inconsistency**

Now, suppose we have the above problem of online/offline mismatch in a live model in production serving tens of thousands of predictions. Arize will help you surface up when such mismatch occurs, and provides you with tool to troubleshoot how inconsistency may have occured.


## Step 4.1 Setting up Project on Arize
**Data Consistency** is an Arize feature that's availiable under the **Projects** tab. To use it, you have to **create a new project** containing your models with the same match environments. For example, all models with same feature schema (such as one instance of a model deployed for a stores/cities/state) can use the same **Projects** page.


![Create Project.png](https://storage.googleapis.com/arize-assets/fixtures/create_project.gif)

## Step 4.2 Setting up Match Environment

Since there may be many validation datasets logged for a model, we will need to select the validation data identifiable by `batch_id`, which in our example above, we labelled `batch_id = 'offline'`.

We want to then set up a match environment. then enter `offline` as our match environment under `validation` as our match environment for `production`. Then, we want to select our model name as show below. 

### 🔁 You can click on the gif to view it again.
[![Set_up_match_config.png](https://storage.googleapis.com/arize-assets/fixtures/data_consistency_environment.gif)](https://storage.googleapis.com/arize-assets/fixtures/data_consistency_environment.gif)

# **Step 5: Arize Platform - Troubleshooting Inconsistency and Why This Matters**

**Note:** The following demo is an expanded example, where the same delayed arrival of information to data source before offline fetching is repeated for tens of thousands of data.

## **Step 5.1: Checking Data Consistency by Feature**
We first want to see the overall data consistency within all of our features. We can see the distribution of any mismatches by clicking into each feature. By toggling on **Show Unmatched Only**, we can see the overall distribution shape and residual of the mismatch.


### 🔁 You can click on the gif to view it again.
[![Feature_view.png](https://storage.googleapis.com/arize-assets/fixtures/data_consistency_feature_view_2.gif)](https://storage.googleapis.com/arize-assets/fixtures/data_consistency_feature_view_2.gif)


## **Step 5.2: Troubleshooting with Heatmap**
Base on the **Distribution Comparisons** and **Residual Error** card, we can already see that there is a online feature distribution mismatch that is one-sided (i.e. all data from get_historical_feature higher than they are on the online environment). We can see that there is a delay, but let's take a deeper dive.

### 🔁 You can click on the gif to view it again.
[![Heatmap_view.png](https://storage.googleapis.com/arize-assets/fixtures/data_consistency_heatmap_view.gif)](https://storage.googleapis.com/arize-assets/fixtures/data_consistency_heatmap_view.gif)

From investigating the heatmap and adjusting the max intensity, we can validate that the mismatch is one sided, suggesting a delay in data rolling up to the feature store.

# **Step 5.3: Why does this matter**
We always want our models to have the best performance possible. In this case, the offline example seems to be skewed due to the delayed arrival of data in our system. This will have a negative impact on model performance due to training/production distribution difference, which we will want to eliminate as much as possible.

# **Overview**


Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.